In [1]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
data = pd.read_csv("./churn.csv")
data.drop("RowNumber", inplace=True, axis=1)
data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 9.1. Каково соотношение ушедших и лояльных клиентов? Покажите это на графике и дайте комментарий по соотношению.

In [3]:
data_croped = data[["Exited"]].copy()
data_croped["Exited"] = data_croped["Exited"].apply(lambda x: "Лояльные" if x else "Ушедшие").astype("string")

fig = px.histogram(
    data_croped,
    x="Exited",
    color="Exited",
    width=7e2,
)
fig.update_layout(
    title="Cоотношение ушедших и лояльных клиентов",
    xaxis_title="Клиенты",
    yaxis_title="Кол-во Клиентов",
    legend_title="Лояльность",
)
fig.show()

Как видно из графика выше, количество лояльных клиентов(2037) в четыре раза меньше, чем ушедших(7963). Довольно плохие показатели.

## 9.2. Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. Опишите распределение и сделайте выводы.

In [4]:
data_croped = data[["Balance"]].copy()

fig = px.histogram(
    data_croped[(data_croped["Balance"] > 2500)]["Balance"],
    x="Balance",
    width=7e2,
)
fig.update_layout(
    title="Распределение баланса пользователей",
    xaxis_title="Баланс",
    yaxis_title="Кол-во Клиентов",
)
fig.show()

Как видно из графика выше, распределение баланса пользователей имеет нормальную форму с небольшими выбросами на 250к и 2.5к. По графику можно сделать вывод, что наибольшее кол-во ушедших и лояльных клиентов имеет баланс в диапазоне от 100к до 140к. 

## 9.3. Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [5]:
data_croped = data[["Exited", "Balance"]].copy()
data_croped["Exited"] = data_croped["Exited"].apply(lambda x: "Лояльные" if x else "Ушедшие").astype("string")

fig = px.box(
    data_croped,
    x="Exited",
    y="Balance",
    color="Exited",
    width=7e2,
)
fig.update_layout(
    title="Распределение баланса клиента в разрезе признака оттока",
    xaxis_title="Лояльность",
    yaxis_title="Баланс",
    legend_title="Лояльность",
)
fig.show()

In [6]:
bins = list(range(0, 250_000, 2000))
data_gr = data.groupby(pd.cut(data["Balance"], bins))
data_k = data_gr[["Exited"]].mean()
data_k = data_k.fillna(0)

fig = px.scatter(
    data_k,
    y="Exited",
    x=list(range(0, 248, 2)),
    width=8e2
)
fig.update_layout(
    title="Отношение баланса к проценту ушедших",
    xaxis_title="Баланс в тыс.",
    yaxis_title="Процент ушедших / 100%",
)

fig.show()

Как видно из графиков выше, у лояльных клиентов по медиане больше денег на балансе, чем у ушедших. Также можно заметить, что большое кол-во клиентов с нулевым балансом ушли, что может говорить о том, что им не понравились условия, банк брал большие проценты за вклады, либо они просто не могли положить деньги. Еще можно заметить, что некоторое кол-во клиентов с большим балансом ушли, что может указывать на смену условий по вклады на невыгодные либо обложением баланса процентом.

## 9.4. Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [7]:
data_croped = data[["Exited", "Age"]].copy()
data_croped["Exited"] = data_croped["Exited"].apply(lambda x: "Лояльные" if x else "Ушедшие").astype("string")

fig = px.box(
    data_croped,
    x="Exited",
    y="Age",
    color="Exited",
    width=7e2,
)
fig.update_layout(
    title="Распределение возраста в разрезе признака оттока",
    xaxis_title="Лояльность",
    yaxis_title="Возраст",
    legend_title="Лояльность",
)
fig.show()

В группе ушедших клиентов больше всего выбросов, начиная с 56 лет и заканчивая 92 годами. Медианный возраст ушедших клиентов равняется 36 годам - более молодые и менее обеспеченные клиенты чаще покидают этот банк. Но если посмотреть на лояльных клиентов, то можно понять, что больше всего остаются клиенты в возрасте от 38 до 51 года, и банку стоит обратить внимание на этот возрастной промежуток.

## 9.5. Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

In [8]:
data_croped = data[["Balance", "CreditScore", "Exited"]].copy()
data_croped["Exited"] = data_croped["Exited"].apply(lambda x: "Лояльные" if x else "Ушедшие").astype("string")

fig = px.scatter(
    data_croped,
    x="Balance",
    y="CreditScore",
    color="Exited",
    width=7e2,
)
fig.update_layout(
    title="Взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты",
    xaxis_title="Баланс",
    yaxis_title="Кредитный рейтинга",
    legend_title="Лояльность",
)
fig.show()

Как видно из графика выше, четкой взаимосвязи между признаками не прослеживается. Однако можно заметить, что у ушедших клиентов с балансом 0 полный спектр кредитного рейтинга. Это говорит о том, что клиенты выводили все деньги перед тем, как уйти, либо вообще не вносили денег и открыли второй аккаунт в этом банке, но он им не понравился. Также есть ушедшие клиенты с ненулевым балансом и высоким кредитным ретийнгом, что может говорить о том, что данные были взяты до того, как клиенты смогли вывести деньги

## 9.6 Кто чаще уходит, мужчины или женщины? Постройте график, который иллюстрирует это.

In [9]:
data_croped = data[["Exited", "Gender"]].copy()
data_croped = data_croped[(data_croped["Exited"] == 1)]
data_croped["Gender"] = data_croped["Gender"].apply(lambda gen: "Мужской" if gen == "Male" else "Женский")

fig = px.pie(
    data_croped,
    values="Exited",
    names="Gender",
    width=7e2,
)
fig.update_layout(
    title="Процент ушедших клиентов по полу",
    legend_title="Пол", 
)
fig.show()

По графику выше можно сказать, что женщины уходят чаще, чем мужчины

## 9.7. Как отток клиентов зависит от числа приобретённых у банка услуг? Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

In [10]:
data_croped = data[["Exited", "NumOfProducts"]].copy()

data_gr = data_croped.groupby(["NumOfProducts", "Exited"]).agg({"Exited": {"count"}})
data_gr.columns = ["exited_count"]
data_gr = data_gr.reset_index()
data_gr["Exited"] = data_gr["Exited"].apply(lambda x: "Лояльные" if x else "Ушедшие").astype("string")
data_gr = data_gr.sort_values(by="Exited")

fig = px.bar(
    data_gr,
    x="NumOfProducts",
    y="exited_count",
    color="Exited",
    width=7e2,
    barmode = 'group'
)
fig.update_layout(
    title="Отток клиентов от числа приобретённых у банка услуг",
    xaxis_title="Кол-во услуг",
    yaxis_title="Кол-во клиентов",
    legend_title="Лояльность",
)
fig.show()

По графику можно судить, что отток клиентов зависит от кол-во услуг. Клиенты, которые воспользовались одной услугой могли решить, что условия им не нравятся, и уйти. Больше всего клиентов воспользовались 1 или 2 услугами и кто-то решил остаться, а кому-то не понравилось, и они просто ушли. Видно, что получив вторую услугу, уходят больше, чем те, которые взяли лишь одну

## 9.8. Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это. Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

In [11]:
data_croped = data[["IsActiveMember", "Exited"]].copy()

data_gr = data_croped.groupby(["IsActiveMember", "Exited"]).agg({"Exited": {"count"}})
data_gr.columns = ["exited_count"]
data_gr = data_gr.reset_index()
data_gr["Exited"] = data_gr["Exited"].apply(lambda x: "Лояльные" if x else "Ушедшие").astype("string")
data_gr["IsActiveMember"] = data_gr["IsActiveMember"].apply(lambda x: "Активный" if x else "Не активный").astype("string")
data_gr = data_gr.sort_values(by="Exited")

fig = px.bar(
    data_gr,
    x="IsActiveMember",
    y="exited_count",
    color="Exited",
    width=7e2,
    barmode = 'group'
)
fig.update_layout(
    title="Зависимось статуса активного клиента от оттока клиентов",
    xaxis_title="Активность",
    yaxis_title="Кол-во Клиентов",
    legend_title="Лояльность",
)
fig.show()

Данному банк может ввести систему акции и бенефитов неактивным клиентам, чтобы повысить активность и привлечь новых пользователей. Также банк может создать реферальную систему для всех клиентов с бонусами, что может помочь с удержанием.

## 9.9. В какой стране доля ушедших клиентов больше? Постройте тепловую картограмму, которая покажет это соотношение на карте мира. Предположите, с чем это может быть связано.

In [12]:
data_croped = data[["Geography", "Exited"]].copy()
data_gr = data_croped.groupby("Geography", as_index=False)["Exited"].mean()
data_gr.columns = ["Geography","mean"]

fig = px.choropleth(
    data_gr,
    locations="Geography",
    locationmode = "country names",
    color="mean",
    width=8e2,
)
fig.update_layout(
    title="Доля ушедших клиентов по странам",
    legend_title="Лояльность",
)

fig.show()

В данной выборе всего три страны: Германия, Испания и Франция. Из графика видно, что наибольший процент ушедших клиентов в Германии. Это может быть связано с менталитетом. Также, возможно, в Германии есть банки с лучшим набором предложений и условий. 

## 9.10. На основе полученной сводной таблицы постройте тепловую карту с аннотацией. Найдите на тепловой карте категории клиентов, которые уходят чаще всего.

In [13]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"
    
data_croped = data[["CreditScore", "Exited", "Tenure"]].copy()
data_croped["CreditScoreCat"] = data_croped["CreditScore"].apply(get_credit_score_cat)
table = pd.pivot_table(data_croped, values="Exited", index=["CreditScoreCat"], columns=["Tenure"], aggfunc="mean")

fig = px.imshow(table, width=8e2)
fig.update_layout(
    title="Зависимость оттока от кредитного рейтинга и длительностью пользования",
    xaxis_title="Длительность пользования",
    yaxis_title="Кредитный рейтинг",
)
fig.show()

Из графика выше видно, что чаще уходят клиенты с низким кредитным рейтингом. Также уходят новые клиенты с отличным рейтингом, которым, скорей всего, не понравились условия. Также можно предположить, что банк менял условия с течением лет, что не нравилось некоторым клиентам, и они уходили